In [1]:
from pathlib import Path

def find_project_root(name_substring: str = "riseholme",
                      required_subpath: Path = Path("Data") / "Normal" / "alltogether",
                      start: Path = Path.cwd()) -> Path:
    """
    Durchläuft start und alle Eltern:
      1. Prüft, ob das aktuelle Verzeichnis selbst passt (Name + required_subpath).
      2. Prüft alle direkten Unterverzeichnisse (Geschwister), ob dort
         name_substring im Namen ist und required_subpath existiert.
    Gibt das gefundene Verzeichnis (Path) zurück oder wirft FileNotFoundError.
    """
    for p in [start] + list(start.parents):
        # 1) Ist p selbst unser Projekt-Root?
        if name_substring.lower() in p.name.lower() and (p / required_subpath).is_dir():
            return p

        # 2) Sonst durchsuche alle Unterordner von p nach unserem Projekt-Root
        for child in p.iterdir():
            if (child.is_dir()
                and name_substring.lower() in child.name.lower()
                and (child / required_subpath).is_dir()):
                return child

    raise FileNotFoundError(f"Kein Verzeichnis gefunden, das '{required_subpath}' enthält "
                            f"und '{name_substring}' im Namen trägt (Beginn: {start})")

# Verwende die neue Funktion
BASE_DIR = find_project_root()
print(f"✅ Projekt-Root gefunden: {BASE_DIR}")

# Jetzt kannst du weitermachen:
image_dir = BASE_DIR / 'Data' / 'Normal' / 'alltogether'
label_dir = BASE_DIR / 'labels' / 'alltogether'
label_dir.mkdir(parents=True, exist_ok=True)

class_id = 0  # Klasse 0 = "ripe"
count = 0

for img_path in image_dir.iterdir():
    if img_path.suffix.lower() in {'.jpg', '.jpeg', '.png'}:
        txt_path = label_dir / f"{img_path.stem}.txt"
        with open(txt_path, 'w') as f:
            f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")
        count += 1

print(f"✅ {count} Labels erstellt in {label_dir}")


✅ Projekt-Root gefunden: c:\Users\Maxi\Documents\Forschsem\Erdbeeren\Riseholme-2021
✅ 961 Labels erstellt in c:\Users\Maxi\Documents\Forschsem\Erdbeeren\Riseholme-2021\labels\alltogether


datensatz splitten

In [2]:
from pathlib import Path
import sys
!{sys.executable} -m pip install scikit-learn

from sklearn.model_selection import train_test_split
import shutil


def find_project_root(required_subpath: Path = Path("Data") / "Normal" / "alltogether",
                      start: Path = Path.cwd()) -> Path:
    """
    Durchläuft start und alle Eltern:
      1. Prüft, ob das aktuelle Verzeichnis selbst required_subpath enthält.
      2. Durchsucht alle Unterordner jedes Eltern-Verzeichnisses nach required_subpath.
    Gibt das erste gefundene Verzeichnis zurück oder wirft FileNotFoundError.
    """
    for p in [start] + list(start.parents):
        # 1) Passt p selbst?
        if (p / required_subpath).is_dir():
            return p
        # 2) Oder eine seiner Unterordner?
        for child in p.iterdir():
            if child.is_dir() and (child / required_subpath).is_dir():
                return child
    raise FileNotFoundError(f"Kein Verzeichnis gefunden, das '{required_subpath}' enthält (Beginn: {start})")

# 1) Projekt-Root ermitteln
BASE_DIR = find_project_root()
print(f"✅ Projekt-Root gefunden: {BASE_DIR}")

# 2) Quell- und Ziel-Pfade definieren
IMG_SRC   = BASE_DIR / 'Data'    / 'Normal' / 'alltogether'
LBL_SRC   = BASE_DIR / 'labels'  / 'alltogether'

IMG_TRAIN = BASE_DIR / 'images'  / 'train'
IMG_VAL   = BASE_DIR / 'images'  / 'val'
LBL_TRAIN = BASE_DIR / 'labels'  / 'train'
LBL_VAL   = BASE_DIR / 'labels'  / 'val'

# 3) Zielordner anlegen
for d in (IMG_TRAIN, IMG_VAL, LBL_TRAIN, LBL_VAL):
    d.mkdir(parents=True, exist_ok=True)

# 4) Dateinamen (Stem) sammeln und splitten
files = [f.stem for f in IMG_SRC.iterdir() if f.suffix.lower() in {'.png', '.jpg', '.jpeg'}]
train, val = train_test_split(files, test_size=0.2, random_state=42)

# 5) Kopieren
for f in train:
    # Bild-Extension automatisch prüfen
    src_img = next(IMG_SRC.glob(f"{f}.*"))
    shutil.copy(src_img, IMG_TRAIN / src_img.name)

    src_lbl = LBL_SRC / f"{f}.txt"
    shutil.copy(src_lbl, LBL_TRAIN / src_lbl.name)

for f in val:
    src_img = next(IMG_SRC.glob(f"{f}.*"))
    shutil.copy(src_img, IMG_VAL / src_img.name)

    src_lbl = LBL_SRC / f"{f}.txt"
    shutil.copy(src_lbl, LBL_VAL / src_lbl.name)

print(f"✅ Train: {len(train)} | Val: {len(val)} Dateien kopiert")


✅ Projekt-Root gefunden: c:\Users\Maxi\Documents\Forschsem\Erdbeeren\Riseholme-2021
✅ Train: 768 | Val: 193 Dateien kopiert


In [5]:
import sys
!{sys.executable} -m pip install ultralytics

from ultralytics import YOLO
model = YOLO('yolov8n.pt')             # oder yolov8s.pt, -m.pt etc.
model.train(
    data='data.yaml',                  # deine Konfig
    epochs=5,                         # je nach Bedarf , 25 reichen (1h)
    imgsz=640,
    batch=16,
    augment=True,
    project='runs/train',              # Ausgabeordner
    name='alltogether_brrr'                   # Lauf-Name
)


Ultralytics 8.3.120  Python-3.13.3 torch-2.7.0+cpu CPU (AMD Ryzen 5 2600 Six-Core Processor)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs/train, name=alltogether_brrr3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, li

train: Scanning C:\Users\Maxi\Documents\Forschsem\Erdbeeren\Riseholme-2021\labels\train.cache... 840 images, 0 backgrounds, 0 corrupt: 100%|██████████| 840/840 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.10.0 ms, read: 43.712.8 MB/s, size: 9.0 KB)



c:\Users\Maxi\anaconda3\envs\Erdbeerenv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\Maxi\Documents\Forschsem\Erdbeeren\Riseholme-2021\labels\val.cache... 261 images, 0 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]

Plotting labels to runs\train\alltogether_brrr3\labels.jpg... 


...

                   all        261        261          1          1      0.995      0.995



5 epochs completed in 0.436 hours.
Optimizer stripped from runs\train\alltogether_brrr3\weights\last.pt, 6.2MB
Optimizer stripped from runs\train\alltogether_brrr3\weights\best.pt, 6.2MB

Validating runs\train\alltogether_brrr3\weights\best.pt...
Ultralytics 8.3.120  Python-3.13.3 torch-2.7.0+cpu CPU (AMD Ryzen 5 2600 Six-Core Processor)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:52<00:00,  5.89s/it]


                   all        261        261          1          1      0.995      0.995
Speed: 1.4ms preprocess, 193.5ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs\train\alltogether_brrr3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000029788C87460>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480